# BEACON+seqARIMA User Manual
## CH1 - Quick Start: Data Cleaning with seqARIMA in BEACON
BEACON is a fully data-driven pipeline designed to detect unmodeled gravitational wave (GW) transients. By combining sequential autoregressive modeling (seqARIMA) and anomaly clustering, BEACON provides a low-latency and model-agnostic framework for robust burst detection.

Below are the quick start steps you need to follow for performing data cleaning by seqARIMA.

### Installation
1. System requirement
   - Python >= 3.10
   - C compiler
2. Dependencies
   - numpy
   - scipy
   - matplotlib
   - pycbc
3. Installation

In [ ]:
git clone https://github.com/OddThumb/beacon-py.git  
cd beacon-py
pip install .

### Load Data
We can load data either from local or from pyCBC. Here we take an example of GW150914.
1. From local

In [ ]:
bc.IO.read_H5('path/to/GW150914.hdf5', sampling_freq={sampling_frequency})

2. From `pyCBC`

In [ ]:
from pycbc.catalog import Merger
import beacon.seqARIMA as arm
merger = Merger("GW150914")
tgps = merger.time
strain_H1_pycbc = merger.strain(ifo="H1")
strain_L1_pycbc = merger.strain(ifo="L1")

Time series objects could be created by the `ts` class of `beacon-py`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import beacon as bc

strain_H1_ts = bc.ts.from_pycbc(strain_H1_pycbc)
strain_L1_ts = bc.ts.from_pycbc(strain_L1_pycbc)

## Data cleaning with seqARIMA

seqARIMA is "sequential AutoRegressive Integrated Moving Average" model for subtracting instrumental noise in GW data by following ***sequential*** 4 stages:

[Stage name (`parameter name`): description]
1. Difference (`d`): parameter for stationarity. This represents the orders of the difference filter, which defines how much the strain changed from the previous moment. 
   - option: d=0 (no differencing process), or d='auto' (automatic fitting).
2. Autoregressive (AR) model residual (`p`): parameter for fitting time domain AR model. This represents the maximum order of AR model for searching the optimal model.  
   - option: integer; Recommand value is 4000~8000. 
3. Moving Average (MA) smoothing (`q`): smoothing parameter for utilizing MA smoothing filter. If q is an integer, a single MA filer will be performed, which higher p provides smoother filter; if q is insert as a integer array, an Ensemble of Average will be performed. 
   - option: integer or integer array.
4. Bandpass filter (`fl`, `fu`): constrains frequency range. According to user's interest of frequency range, `fl` is lower frequency bound and `fu` is upper frequency bound.  
   - option: float or None.

In [ ]:
arm_H1 = arm.seqarima(
    strain_H1_ts,
    d="auto",
    p=5000,  # automatic search within this upper bound. e.g. optimal order p <= 4000,
    q=range(1, 21),  # implementing "ensemble of average" rather than single MA filter,
    fl=32,
    fu=512, verbose=True
)

print(f"Difference stage meta info: \n {arm_H1.diff_meta}")
print(f"AR stage meta info: \n {arm_H1.ar_meta}")
print(f"EoA stage meta info: \n {arm_H1.ma_meta}")
print(f"BP stage meta info: \n {arm_H1.bp_meta}")

In [ ]:
arm_L1 = arm.seqarima(
    strain_L1_ts,
    d="auto",
    p=5000,  # automatic search within this upper bound. e.g. optimal order p <= 4000,
    q=range(1, 21),  # implementing "ensemble of average" rather than single MA filter,
    fl=32,
    fu=512,
)

print(f"Difference stage meta info: \n {arm_L1.diff_meta}")
print(f"AR stage meta info: \n {arm_L1.ar_meta}")
print(f"EoA stage meta info: \n {arm_L1.ma_meta}")
print(f"BP stage meta info: \n {arm_L1.bp_meta}")

BEACON implemented plotting functions, let try to plot them.
- Timeserires

In [ ]:
fig, ax = plt.subplots()
bc.plot.plot_oscillo(arm_H1, tzero=tgps, alpha=0.7, ax=ax, label="H1")
bc.plot.plot_oscillo(arm_L1, tzero=tgps, alpha=0.7, ax=ax, label="L1")
plt.legend()
plt.show()

# Zoom-in around (-0.2, +0.1) + tgps
fig, ax = plt.subplots()
bc.plot.plot_oscillo(
    arm_H1, tzero=tgps, trange=(tgps - 0.2, tgps + 0.1), lw=1, ax=ax, label="H1"
)
bc.plot.plot_oscillo(
    arm_L1, tzero=tgps, trange=(tgps - 0.2, tgps + 0.1), lw=1, ax=ax, label="L1"
)
plt.legend()
plt.show()

- Spectrograms

In [ ]:
bc.plot.plot_spectro(
    arm_H1,
    tzero=tgps,
    trange=(tgps - 0.2, tgps + 0.1),
    title="seqARIMA-denoised GW150914 (H1)",
)
plt.show()  

In [ ]:
bc.plot.plot_spectro(
    arm_H1,
    tzero=tgps,
    trange=(tgps - 0.2, tgps + 0.1),
    title="seqARIMA-denoised GW150914 (H1)",
)
plt.show()

- `ts` class
  - duration
  - start
  - end
  - times
  - length
  - window
  - to_pycbc
  - from_pycbc
  - to_gwpy
  - from_gwpy
  
- `plot` class
  - plot_spectro: pycbc's q trans
  - plot_oscillo